# 자연어 처리와 토큰화, 수치화, 수량화에 의한 정보요약

인간의 언어를 컴퓨터와 같은 기계가 묘사할 수 있도록 연구하고 구현하는 분야

: 음성 - Text화 - 말뭉치 사전 기준 토큰화 - 입력벡터 - 딥러닝 모델 처리 - 출력벡터 - 말뭉치 사전 참조 Text 벡터화 - 음성화

# 09.1 자연어 처리에 의한 텍스트 데이터의 상호변환

텍스트를 음성으로 변환 라이브러리 : https://pypi.org/project/gTTS/

음성인식 라이브러리 : https://pypi.org/project/SpeechRecognition/

## 9.1.1 음성과 텍스트의 상호변환을 위한 라이브러리 설치

In [1]:
# 필요한 라이브러리 설치
!pip install gTTS SpeechRecognition pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 26.3 MB/s eta 0:00:00


## 9.1.2 텍스트를 음성으로 젼환하는 gTTS 기술

In [2]:
%%writefile sample.txt
안녕하세요?
저는 인공지능 스피커 입니다.
오늘 날씨 어떤가요?

Writing sample.txt


In [3]:
# 텍스트 파일을 읽어들여 mp3파일로 변환
from gtts import gTTS
with open('sample.txt','r') as f:
    text =f.read()
tts = gTTS(text=text, lang='ko')
tts.save("my_audio.mp3")

In [4]:
# 음성으로 출력
from IPython.display import Audio, display
display(Audio('my_audio.mp3', autoplay=True))

## 9.1.3 음성을 텍스트로 변환하는 STT 기술

In [5]:
# SpeechRecognition 라이브러리는 wav 파일만 인식하여 mp3를 wav파일로 변환이 필요
import pydub
sound = pydub.AudioSegment.from_mp3('my_audio.mp3')
sound.export('my_audio.wav', format="wav")

<_io.BufferedRandom name='my_audio.wav'>

In [6]:
# wav 파일을 텍스트로 변환하여 출력
import speech_recognition as sr
r = sr.Recognizer()
with sr.AudioFile('my_audio.wav') as f:
    audio = r.record(f)
print(r.recognize_google(audio, language='ko'))

안녕하세요 저는 인공지능 스피커입니다 오늘 날씨 어떤가요


## 9.1.4 영상을 텍스트로 변환하는 기술로 확장

In [7]:
# 유튜브 영상 다운로드 및 오디오 추출 라이브러리
!pip install pytube moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00


In [8]:
# 유튜브 영상 다운로드 및 오디오 추출
from pytube import YouTube
from moviepy.editor import VideoFileClip
url = "https://youtu.be/lUkiP31l-jQ"
file ="audio_file"
print("영상을 다운로드 중입니다. 잠시만 기다려 주세요~")
# Download the video
yt = YouTube(url)
stream = yt.streams.get_highest_resolution()
stream.download(filename=file+".mp4")
print("영상다운로드를 완료하고 음성변환 중입니다. 잠시만 더 기다리세요~")
# # Extract the audio
video = VideoFileClip(file+'.mp4')
audio = video.audio
audio.write_audiofile(file+'.mp3')

영상을 다운로드 중입니다. 잠시만 기다려 주세요~
영상다운로드를 완료하고 음성변환 중입니다. 잠시만 더 기다리세요~
MoviePy - Writing audio in audio_file.mp3


MoviePy - Done.


In [9]:
# SpeechRecognition 라이브러리는 wav 파일만 인식하여 mp3를 wav파일로 변환이 필요
import pydub
sound = pydub.AudioSegment.from_mp3('audio_file.mp3')
sound.export('wav_file.wav', format="wav")

<_io.BufferedRandom name='wav_file.wav'>

In [10]:
# 긴문장을 세그멘테이션하여 텍스트화하는 함수 정의
# transcribe_audio(path): 텍스트로 변환하는 함수,
# get_large_audio_transcription_on_silence(path): 오디오 파일 세그멘테이션 함수
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
r = sr.Recognizer()
def transcribe_audio(path):
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        text = r.recognize_google(audio_listened, language='ko')
    return text

def get_large_audio_transcription_on_silence(path):
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound,
        min_silence_len = 500,
        silence_thresh = sound.dBFS-14,
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        try:
            text = transcribe_audio(chunk_filename)
        except sr.UnknownValueError as e:
            print("Error:", str(e))
        else:
            text = f"{text.capitalize()}. "
            print(chunk_filename, ":", text)
            whole_text += text
    return whole_text

In [11]:
# wav 파일 입력 text_file 출력
path = "wav_file.wav"
text_file = get_large_audio_transcription_on_silence(path)
print("\nFull text:", text_file)

audio-chunks/chunk1.wav : 안녕하세요 에이제트 테크놀로지다 오늘은 블렌더에서 매우 유용한 블렌더 gis 애드온에 대해서 알아보겠습니다 구글에서 블렌더 gis 검색한 다음에. 
audio-chunks/chunk2.wav : 다음과 같이 기타부 켜서 코드 다운로드 지불해서 블렌더 gis 다운 받습니다 그리고 블렌더를 실행한 다음에. 
audio-chunks/chunk3.wav : 에. 
audio-chunks/chunk4.wav : Preference. 
audio-chunks/chunk5.wav : 애드온에서. 
audio-chunks/chunk6.wav : 방금 다운 받았던. 
audio-chunks/chunk7.wav : Gis 선택해 줍니다 인스톨 애드온을 선택하고 여기 3d view 블렌더 gis 체크해 줍니다 그리고 밑에 내려 보면은 지금 캐시 폴더를 저장해 줘야 되는데요 3d 데이터를 제대로 다운 받을 경우에 용량이 상당히 많기 때문에 원하는 곳을 지정해 주시면 됩니다 이렇게 해서 블렌더 gis 사용할 준비가 됐습니다 여기에 gis 추가가 되었습니다 gis 사용하는 방법은 간단합니다 먼저이 디폴트 큐브는 delete 키를 눌러서 지우겠습니다. 
audio-chunks/chunk8.wav : Gis 웹 지오 데이터 베이스 맵을 선택합니다 그리고 소스 레이어가 기본으로 선택되어 있는데요이 상태에서 오케이를 선택해 줍니다 그러면 이렇게 구글 지도가 나오는데요 마우스 휠 키를 조절하고 마우스 가운데 버튼을 눌러서 이렇게 지도를 조절할 수가 있습니다 원하는 곳으로 움직인 다음에 기다리고 있으면. 
audio-chunks/chunk9.wav : 베키 왼쪽에 다운로드가 됐고 점점 더 정확하게 위치가 나타나는 것을 볼 수가 있습니다 그리고 단축키 쥐를 누르고 검색을 해서도 찾을 수가 있습니다. 
audio-chunks/chunk10.wav : 런던을 누르고 줌 레벨은 15로 하겠습니다. 
audio-chunks/chunk11.wav : 그러면 이

In [12]:
text_file

'안녕하세요 에이제트 테크놀로지다 오늘은 블렌더에서 매우 유용한 블렌더 gis 애드온에 대해서 알아보겠습니다 구글에서 블렌더 gis 검색한 다음에. 다음과 같이 기타부 켜서 코드 다운로드 지불해서 블렌더 gis 다운 받습니다 그리고 블렌더를 실행한 다음에. 에. Preference. 애드온에서. 방금 다운 받았던. Gis 선택해 줍니다 인스톨 애드온을 선택하고 여기 3d view 블렌더 gis 체크해 줍니다 그리고 밑에 내려 보면은 지금 캐시 폴더를 저장해 줘야 되는데요 3d 데이터를 제대로 다운 받을 경우에 용량이 상당히 많기 때문에 원하는 곳을 지정해 주시면 됩니다 이렇게 해서 블렌더 gis 사용할 준비가 됐습니다 여기에 gis 추가가 되었습니다 gis 사용하는 방법은 간단합니다 먼저이 디폴트 큐브는 delete 키를 눌러서 지우겠습니다. Gis 웹 지오 데이터 베이스 맵을 선택합니다 그리고 소스 레이어가 기본으로 선택되어 있는데요이 상태에서 오케이를 선택해 줍니다 그러면 이렇게 구글 지도가 나오는데요 마우스 휠 키를 조절하고 마우스 가운데 버튼을 눌러서 이렇게 지도를 조절할 수가 있습니다 원하는 곳으로 움직인 다음에 기다리고 있으면. 베키 왼쪽에 다운로드가 됐고 점점 더 정확하게 위치가 나타나는 것을 볼 수가 있습니다 그리고 단축키 쥐를 누르고 검색을 해서도 찾을 수가 있습니다. 런던을 누르고 줌 레벨은 15로 하겠습니다. 그러면 이렇게 런던 지역이 블렌더에 나타나는데요 마우스 왼쪽 버튼을 누르고 화면을 움직여 보면은. 지도가 나타나는 것을 볼 수 있습니다 이때이 상태에서 단축키 2를 누르면은. 런던 지역이. 블렌더에 이렇게 지도로 들어오는 것을 볼 수가 있습니다 그리고 이제이 2d 지도에 높이를 나타내면서 건물을 세워 줘야 됩니다 그렇게 하기 위해서는 먼저 gis. 웹 지오 데이터에서. 겟 엘리베이션 srtm 선택해 줍니다 그리고 오케이를 선택해 주면은 이렇게 블렌더에 나오는 지도에 미세한 높이가 표시되는 것을 볼 수가 있습니다 지금 높이가 잘 안 보일

# 09.2 텍스트 데이터의 토큰화 방법

토큰화 : 일정한 입력단위로 분할

## 9.2.1 문자단위, 띄어쓰기단위, 문장단위 토큰화

In [13]:
corpus = """나는 학교에 다니는 학생입니다.
나는 좋은 선생님입니다.
당신은 매우 좋은 선생님입니다."""
corpus

'나는 학교에 다니는 학생입니다.\n나는 좋은 선생님입니다.\n당신은 매우 좋은 선생님입니다.'

In [14]:
# 여러가기 토큰화 방법 : 문자단위, 띄어쓰기 단위, 문장단위 토큰화
print(list(corpus))
# space 기준 단어단위 나누기 : split
print(corpus.split())
# 문장단위는 개행 escape 문자 \n 기준으로 나누기
print(corpus.split("\n"))

['나', '는', ' ', '학', '교', '에', ' ', '다', '니', '는', ' ', '학', '생', '입', '니', '다', '.', '\n', '나', '는', ' ', '좋', '은', ' ', '선', '생', '님', '입', '니', '다', '.', '\n', '당', '신', '은', ' ', '매', '우', ' ', '좋', '은', ' ', '선', '생', '님', '입', '니', '다', '.']
['나는', '학교에', '다니는', '학생입니다.', '나는', '좋은', '선생님입니다.', '당신은', '매우', '좋은', '선생님입니다.']
['나는 학교에 다니는 학생입니다.', '나는 좋은 선생님입니다.', '당신은 매우 좋은 선생님입니다.']


## 9.2.2 한글현태소 분석기를 이용한 토큰화

In [15]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 35.4 MB/s eta 0:00:00


In [16]:
# 형태소 단위 토큰화
import konlpy
komoran = konlpy.tag.Komoran()
print(komoran.morphs(corpus))
print(komoran.nouns(corpus))

['나', '는', '학교', '에', '다니', '는', '학생', '이', 'ㅂ니다', '.', '나', '는', '좋', '은', '선생님', '이', 'ㅂ니다', '.', '당신', '은', '매우', '좋', '은', '선생님', '이', 'ㅂ니다', '.']
['학교', '학생', '선생님', '선생님']


## 9.2.3 기존에 만들어진 단어사전을 기준으로한 토큰화

In [17]:
# 기존에 만들어진 단어사전 활용
import sentencepiece as spm
!wget https://github.com/byungjooyoo/Dataset/raw/main/kowiki_32000.model
sp = spm.SentencePieceProcessor("kowiki_32000.model")
print(sp.encode_as_pieces(corpus))

--2024-04-15 22:12:34--  https://github.com/byungjooyoo/Dataset/raw/main/kowiki_32000.model
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/byungjooyoo/Dataset/main/kowiki_32000.model [following]
--2024-04-15 22:12:34--  https://raw.githubusercontent.com/byungjooyoo/Dataset/main/kowiki_32000.model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 814744 (796K) [application/octet-stream]
Saving to: ‘kowiki_32000.model’

kowiki_32000.model  100%[===================>] 795.65K  --.-KB/s    in 0.06s   

2024-04-15 22:12:34 (12.6 MB/s) - ‘kowiki_32000.model’ saved [814744/814744]

['▁나는', '▁학교에', '▁다니는'

In [18]:
# 단어사전의 아이디로 수치화, 수치화된 것을 본래의 문장으로 복원
print(sp.encode(corpus))
print(sp.decode_pieces(sp.encode(corpus)))

[3782, 10732, 10832, 1782, 5863, 7, 3782, 1229, 14024, 5863, 7, 7301, 18, 446, 1229, 14024, 5863, 7]
나는 학교에 다니는 학생입니다. 나는 좋은 선생님입니다. 당신은 매우 좋은 선생님입니다.


## 9.2.4 사전학습 모델에 의한 토큰화

참조 : https://huggingface.co/google-bert/bert-base-multilingual-cased

이정범님의 깃허브 참조 https://github.com/Beomi/KcELECTRA

In [20]:
# Bert 다국어 버전의 토큰나이저 활용
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
print(tokenizer.tokenize(corpus))
print(tokenizer.encode(corpus))
print(tokenizer.decode(tokenizer.encode(corpus)))

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

['나는', '학', '##교', '##에', '다', '##니', '##는', '학', '##생', '##입', '##니다', '.', '나는', '좋은', '선', '##생', '##님', '##입', '##니다', '.', '당', '##신', '##은', '매우', '좋은', '선', '##생', '##님', '##입', '##니다', '.']
[101, 100585, 9953, 25242, 10530, 9056, 25503, 11018, 9953, 24017, 58303, 48345, 119, 100585, 79633, 9428, 24017, 108578, 58303, 48345, 119, 9067, 25387, 10892, 42608, 79633, 9428, 24017, 108578, 58303, 48345, 119, 102]
[CLS] 나는 학교에 다니는 학생입니다. 나는 좋은 선생님입니다. 당신은 매우 좋은 선생님입니다. [SEP]


In [21]:
# 이정범님의 사전학습 모델을 이용한 토큰화 : 참조 https://github.com/Beomi/KcELECTRA
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")
print(tokenizer.tokenize(corpus))
print(tokenizer.encode(corpus))
print(tokenizer.decode(tokenizer.encode(corpus)))

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/450k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

['나는', '학교에', '다니는', '학생', '##입니다', '.', '나는', '좋은', '선생님', '##입니다', '.', '당신은', '매우', '좋은', '선생님', '##입니다', '.']
[2, 8582, 23327, 11071, 9069, 8034, 18, 8582, 8213, 15570, 8034, 18, 9730, 11123, 8213, 15570, 8034, 18, 3]
[CLS] 나는 학교에 다니는 학생입니다. 나는 좋은 선생님입니다. 당신은 매우 좋은 선생님입니다. [SEP]


# 09.3 텍스트 데이터의 수치화 표현 : OHE, BOW

말뭉치 사전 : 자연언어 연구를 위해 특정한 목적을 가지고 언어의 표본을 추출한 집합

Text를 수치화 시키는 방법은 말뭉치 사전을 만들고 그것을 기준으로 One-Hot Encoding과 BOW 방법으로 정보를 요약했으나 이 방법들은 비효율 적이고 정보의 유실이 있어 최근에는 Sentencepiece를 이용하여 수치화 수량화한다.

## 9.3.1 단어사전의 개념

In [22]:
docs = '''오늘은 매우 매우 매우 좋은 날씨 입니다
내일도 매우 좋은 날씨를 기대합니다
오늘은 좋은 날씨 내일도 매우 좋은 날씨 입니다'''
words = docs.split()
print(words)
tokens = list(dict.fromkeys(words))
tokens

['오늘은', '매우', '매우', '매우', '좋은', '날씨', '입니다', '내일도', '매우', '좋은', '날씨를', '기대합니다', '오늘은', '좋은', '날씨', '내일도', '매우', '좋은', '날씨', '입니다']


['오늘은', '매우', '좋은', '날씨', '입니다', '내일도', '날씨를', '기대합니다']

In [23]:
token_to_id = {'[PAD]': 0, '[UNK]': 1}
for token in tokens:
    token_to_id[token] = len(token_to_id)
print(token_to_id)

id_to_token = {_id : token for token, _id in token_to_id.items()}
print(id_to_token)

{'[PAD]': 0, '[UNK]': 1, '오늘은': 2, '매우': 3, '좋은': 4, '날씨': 5, '입니다': 6, '내일도': 7, '날씨를': 8, '기대합니다': 9}
{0: '[PAD]', 1: '[UNK]', 2: '오늘은', 3: '매우', 4: '좋은', 5: '날씨', 6: '입니다', 7: '내일도', 8: '날씨를', 9: '기대합니다'}


In [24]:
# 문장별 수치화된 리스트의 리스트
token_ids = []
line_words = docs.split('\n')
for line_word in line_words:
    token_ids.append([token_to_id[word] for word in line_word.split()])
token_ids


[[2, 3, 3, 3, 4, 5, 6], [7, 3, 4, 8, 9], [2, 4, 5, 7, 3, 4, 5, 6]]

## 9.3.2 원핫인코딩(One-Hot Encoding)

In [25]:
# 하나의 행이 하나의 단어를 표현하는 One-Hot 인코딩
one_hot_encodings = []
for line_token in token_ids:
    print(line_token)
    one_hot_line = []
    for id in line_token:
        one_hot = [0] * len(token_to_id)
        one_hot[id] = 1
        print(id, one_hot)
        one_hot_line.append(one_hot)
    print(one_hot_line)
    one_hot_encodings.append(one_hot_line)


[2, 3, 3, 3, 4, 5, 6]
2 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
4 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
5 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
6 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]
[7, 3, 4, 8, 9]
7 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
4 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
8 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
9 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
[2, 4, 5, 7, 3, 4, 5, 6]
2 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
4 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
5 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
7 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
3 [0, 0, 0, 1, 0, 0, 0,

In [26]:
import pandas as pd
pd.DataFrame(one_hot_encodings[0], columns = list(token_to_id.keys()))


,[PAD],[UNK],오늘은,매우,좋은,날씨,입니다,내일도,날씨를,기대합니다
0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0


In [27]:
import numpy as np
np.argmax(np.array(one_hot_encodings[2]), axis=-1)
print(token_ids[2])
for id in token_ids[2]:
  print(id_to_token[id])


[2, 4, 5, 7, 3, 4, 5, 6]
오늘은
좋은
날씨
내일도
매우
좋은
날씨
입니다


In [28]:
# tf로 원핫인코딩을 하기 위해서는 문장의 길이를 동일하게 조정 : 짧은 문장에 PAD 추가
# 3차원 배열로 shape은 (3,8,10)으로 변환 : 가장 긴 문장인 마지막 문장을 기준으로 [PAD]를 추가
import tensorflow as tf
pad_ids = []
for line in token_ids:
    line = line[:8]
    line += [0] * (8 - len(line))
    pad_ids.append(line)
print(pad_ids)

tf_one_hot_encodings = tf.one_hot(indices=pad_ids, depth=len(token_to_id))
tf_one_hot_encodings


[[2, 3, 3, 3, 4, 5, 6, 0], [7, 3, 4, 8, 9, 0, 0, 0], [2, 4, 5, 7, 3, 4, 5, 6]]


<tf.Tensor: shape=(3, 8, 10), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.,

## 9.3.3 Bag of Words(BOW) 표현

In [29]:
sentences = docs.split("\n")
words = []
for sentence in sentences:
    words.append(sentence.split())
words


[['오늘은', '매우', '매우', '매우', '좋은', '날씨', '입니다'],
 ['내일도', '매우', '좋은', '날씨를', '기대합니다'],
 ['오늘은', '좋은', '날씨', '내일도', '매우', '좋은', '날씨', '입니다']]

In [30]:
bows = []
for line in words:
    print(line)
    bow = [0] * len(token_to_id)
    print(bow)
    for t in line:
        # print(t)
        bow[token_to_id[t]] += 1
        print(t, bow)
    print()
    bows.append(bow)
bows = np.array(bows)
bows


['오늘은', '매우', '매우', '매우', '좋은', '날씨', '입니다']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
오늘은 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
매우 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
매우 [0, 0, 1, 2, 0, 0, 0, 0, 0, 0]
매우 [0, 0, 1, 3, 0, 0, 0, 0, 0, 0]
좋은 [0, 0, 1, 3, 1, 0, 0, 0, 0, 0]
날씨 [0, 0, 1, 3, 1, 1, 0, 0, 0, 0]
입니다 [0, 0, 1, 3, 1, 1, 1, 0, 0, 0]

['내일도', '매우', '좋은', '날씨를', '기대합니다']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
내일도 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
매우 [0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
좋은 [0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
날씨를 [0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
기대합니다 [0, 0, 0, 1, 1, 0, 0, 1, 1, 1]

['오늘은', '좋은', '날씨', '내일도', '매우', '좋은', '날씨', '입니다']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
오늘은 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
좋은 [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
날씨 [0, 0, 1, 0, 1, 1, 0, 0, 0, 0]
내일도 [0, 0, 1, 0, 1, 1, 0, 1, 0, 0]
매우 [0, 0, 1, 1, 1, 1, 0, 1, 0, 0]
좋은 [0, 0, 1, 1, 2, 1, 0, 1, 0, 0]
날씨 [0, 0, 1, 1, 2, 2, 0, 1, 0, 0]
입니다 [0, 0, 1, 1, 2, 2, 1, 1, 0, 0]



array([[0, 0, 1, 3, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 1, 1, 1],
       [0, 0, 1, 1, 2, 2, 1, 1, 0, 0]])

## 9.1.4 빈도수 기반 단어 표현의 한계

In [31]:
# DTM 도표
col = token_to_id.keys()
pd.DataFrame(bows, columns=col)


,[PAD],[UNK],오늘은,매우,좋은,날씨,입니다,내일도,날씨를,기대합니다
0,0,0,1,3,1,1,1,0,0,0
1,0,0,0,1,1,0,0,1,1,1
2,0,0,1,1,2,2,1,1,0,0


# 09.4 문서의 수량화 표현과 요약 : DTM, TF-IDF

## 9.4.1 문서단어행렬(DTM, Document-Term Matrix)

In [32]:
!pip install konlpy
!wget https://raw.githubusercontent.com/byungjooyoo/Dataset/main/news.txt


--2024-04-15 22:13:20--  https://raw.githubusercontent.com/byungjooyoo/Dataset/main/news.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13292 (13K) [text/plain]
Saving to: ‘news.txt’

news.txt            100%[===================>]  12.98K  --.-KB/s    in 0s      

2024-04-15 22:13:20 (27.4 MB/s) - ‘news.txt’ saved [13292/13292]



In [33]:
# 기사를 형태소 분석기로 정리하여 리스트로 만들기
import konlpy
docs = []
with open("news.txt", 'r') as f:
    while True:
        line = f.readline()
        if not line: break
        doc = konlpy.tag.Komoran().morphs(line)
        docs.append(' '.join(doc))
docs


["' 28 일 중소 벤처기업 부가 공개 하 ㄴ ‘ 2019년 기준 중소기업 기본 통계 ’ 를 살펴보 면 울산 중소기업 은 전체 기업 의 99 . 9 % , 중소기업 근로자 는 전체 기업 종사자 의 88 . 4 % 를 차지 하 고 있 는 것 으로 나타나 았 다 . 중소기업 기본 통계 는 매년 통계청 의 기업 통계 등록부 를 토대 로 중기 부가 작성 하 는 자료 다 . 집계 결과 2019 년 기준 울산 중소기업 수 는 12 만 7682 개 로 전국 의 1 . 9 % 를 점유 하 았 다 . 전년 ( 12 만 5352 ) 보다 2330 개 ( 1 . 9 % ) 증가 하 았 다 . 세종 ( 3 만 4608 개 ) 을 제외 하 면 제주 ( 11 만 983 개 ) 에 이어 전국 에서 두 번 째 로 적 은 것 으로 조사 되 었 다 . 특히 관광 도시 제주 ( 5 . 7 % 증가 ) 와 의 격차 도 중소기업 격차 로 1년 새 크 게 좁히 어 지 었 다 . 울산 의 중소기업 인수 는 34 만 4092 명 으로 전국 의 2 . 0 % 를 점유 하 았 다 . 전년 ( 33 만 7241 명 ) 보다 중소기업 인수 는 6851 명 ( 2 . 0 % ) 증가 하 았 다 . 울산 의 중소기업 매출액 은 46 조 948 억 원 으로 전국 의 1 . 7 % 를 점유 하 는데 그치 었 다 . 중소기업 매출액 은 전년 ( 44 조 8664 억 원 ) 보다 1 조 2284 억 원 ( 2 . 7 % ) 증가 하 았 다 . 하 지만 , 중기 매출액 은 산업 도시 울산 의 위상 ( 인구 비중 2 . 2 % ) 은 물론 지역 중소기업 수 비중 ( 1 . 95 % ) 에 도 못 미치 었 다 . 3 대 주력 산업 의 수직 계열 화 구조 에서 벗어나 지 못하 고 있 는 울산 의 중소기업 들 의 성장 이 상대 적 으로 부진 하 다는 방증 으로 풀이 되 ㄴ다 . 지역 중기 매출액 역시 세종 ( 10 조 1367 억 원 ) 을 제외 하 면 제주 ( 27 조 4096 억 원 ) 에 이어 두 번 째 로 적 었 다 .

In [34]:
# DTM 테이블로 정리
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(docs)
vocab = vect.vocabulary_
dtm = vect.transform(docs).toarray()
col = vect.get_feature_names_out()
pd.DataFrame(dtm, columns=col)


,00,02,021,06,10,1000,105,11,116,11월,...,확인,확진,환경,환자,활용,회복,회복세,효과,휘발유,힘입
0,0,0,1,0,1,1,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,1,0,0,0,2,0,0,...,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,3,0
3,0,0,0,0,2,0,0,1,0,0,...,3,4,1,1,1,3,0,0,0,0
4,0,0,0,0,1,0,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# 단어사전의 id로 단어찾기 준비
vocab_words = {v:k for k,v in vocab.items()}
vocab_words[9]

'11월'

In [36]:
# 많이 등장하는 단어로 주제 식별
import numpy as np
dtm_order = np.flip(np.argsort(dtm, axis=-1), axis=-1)
for line in dtm_order:
    str_order = [vocab_words[_id] for _id in line[:10] ]
    print(str_order)


['중소기업', '으로', '울산', '매출액', '전국', '증가', '지역', '여성', '보다', '차지']
['오피스텔', '가격', '상승', '울산', '전세', '전국', '아파트', '으로', '지역', '7월']
['으로', '인하', '정부', '휘발유', '대하', '유류', '관리', '개월', '당정', 'lpg']
['에서', '단계', 'ㄴ다', '으로', '코로나', '제한', '개편', '확진', '마스크', '해제']
['류현진', '체인지업', '에서', '이날', '상대로', '패스트', '삼진', '볼넷', '토론토', '병살타']


In [37]:
# 전체 문서의 정보 요약
count_id = np.flip(np.argsort(dtm.sum(axis=0), axis=-1), axis=-1)
print(count_id[:10])
for id in count_id[:10]:
    print(vocab_words[id], dtm.sum(axis=0)[id])

[486 461 431 572 363 451 108 106 533 269]
으로 32
울산 26
에서 24
중소기업 22
상승 17
오피스텔 17
가격 16
ㄴ다 15
전국 15
류현진 14


## 9.4.2 TF-IDF(Term Frequencey - Inverse Document Frequency)

In [38]:
import numpy as np
tf = dtm
df = np.sum(tf > 0, axis=0)
n_df = tf.shape[0] / df
idf = np.log(n_df)
print(tf.shape, df.shape, n_df.shape, idf.shape)
print('세개의문서에서 3번, 1번 등장횟수별 IDF값', round(np.log(5/3),2), round(np.log(5/1),2))
print([round(num,2) for num in idf])

(5, 692) (692,) (692,) (692,)
세개의문서에서 3번, 1번 등장횟수별 IDF값 0.51 1.61
[1.61, 1.61, 0.92, 1.61, 0.51, 1.61, 1.61, 0.0, 1.61, 1.61, 0.22, 0.92, 1.61, 1.61, 1.61, 0.92, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 0.92, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 0.92, 1.61, 0.92, 1.61, 0.92, 1.61, 1.61, 1.61, 0.92, 0.92, 0.92, 1.61, 1.61, 1.61, 1.61, 1.61, 0.51, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 0.92, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 0.22, 1.61, 1.61, 0.92, 0.22, 0.22, 1.61, 1.61, 1.61, 1.61, 0.0, 1.61, 1.61, 1.61, 1.61, 0.92, 0.92, 0.92, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 0.92, 1.61, 0.92, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 0.51, 1.61, 1.61, 1.61, 1.61, 0.92, 1.61, 1.61, 0.51, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61,

In [39]:
import pandas as pd
tf_idf=tf*idf
pd.DataFrame(tf_idf, columns = col)

,00,02,021,06,10,1000,105,11,116,11월,...,확인,확진,환경,환자,활용,회복,회복세,효과,휘발유,힘입
0,0.000000,0.000000,0.916291,0.000000,0.510826,1.609438,0.000000,0.0,0.000000,1.609438,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.218876,1.609438,0.916291,1.609438,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438,0.000000,0.000000,1.609438
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.609438,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438,4.828314,0.000000
3,0.000000,0.000000,0.000000,0.000000,1.021651,0.000000,0.000000,0.0,0.000000,0.000000,...,4.828314,6.437752,1.609438,1.609438,1.609438,4.828314,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.510826,0.000000,1.609438,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(vocabulary=vocab)
tfidf = tfidfv.fit_transform(docs).toarray()
pd.DataFrame(tfidf, columns=col)

,00,02,021,06,10,1000,105,11,116,11월,...,확인,확진,환경,환자,활용,회복,회복세,효과,휘발유,힘입
0,0.000000,0.000000,0.024558,0.000000,0.020386,0.030439,0.000000,0.029009,0.000000,0.030439,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000
1,0.056249,0.028125,0.022691,0.028125,0.000000,0.000000,0.000000,0.026803,0.000000,0.000000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.028125,0.000000,0.00000,0.028125
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040295,0.084563,0.000000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.000000,0.084563,0.25369,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.054850,0.000000,0.000000,0.019513,0.000000,0.000000,...,0.12285,0.1638,0.04095,0.04095,0.04095,0.12285,0.000000,0.000000,0.00000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.024650,0.000000,0.036807,0.052616,0.000000,0.000000,...,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000


In [41]:
import numpy as np
tfidf_order = np.flip(np.argsort(tfidf, axis=-1), axis=-1)
# tfidf_order
topics =[]
n_top = 10
for line in tfidf_order:
  topic = [vocab_words[x] for x in line[:n_top]]
  topics.append(topic)
df = pd.DataFrame(topics)
df

,0,1,2,3,4,5,6,7,8,9
0,중소기업,매출액,울산,전국,증가,으로,여성,보다,지역,차지
1,오피스텔,가격,상승,울산,전세,아파트,전국,면적,부동산,7월
2,인하,대하,유류,휘발유,정부,경유,부탄,lpg,관리,당정
3,단계,개편,코로나,제한,에서,방역,확진,마스크,일상,해제
4,류현진,체인지업,상대로,볼넷,병살타,삼진,토론토,패스트,에서,던지


# 09.5 코사인 유사도에 의한 문서 유사도 평가

In [42]:
import numpy as np
def cos_sim(A, B):
    return np.dot(A, B)/(np.linalg.norm(A)*np.linalg.norm(B))
print(cos_sim(dtm[0],dtm[1]),cos_sim(dtm[0],dtm[2]), cos_sim(dtm[0],dtm[3]), cos_sim(dtm[0],dtm[4]))
print(cos_sim(tfidf[0],tfidf[1]),cos_sim(tfidf[0],tfidf[2]), cos_sim(tfidf[0],tfidf[3]), cos_sim(tfidf[0],tfidf[4]))

0.24992869224796024 0.13486784062821366 0.19127542896706215 0.07643584558374635
0.13967511316579612 0.041630823790375936 0.07295507073539963 0.028434352099514188
